#Samer Baslan
#CMPE-297: Assignment 3 Part 3
#BERT Language Model Fine Tuning

Resources: <br />
https://github.com/huggingface/notebooks/blob/master/examples/language_modeling.ipynb

##Installs

In [1]:
!pip install datasets transformers seqeval
!pip install gradio

     |████████████████████████████████| 290 kB 5.1 MB/s 
     |████████████████████████████████| 3.1 MB 58.1 MB/s 
     |████████████████████████████████| 43 kB 2.7 MB/s 
     |████████████████████████████████| 125 kB 68.7 MB/s 
     |████████████████████████████████| 56 kB 6.4 MB/s 
     |████████████████████████████████| 243 kB 75.2 MB/s 
     |████████████████████████████████| 1.3 MB 68.7 MB/s 
     |████████████████████████████████| 895 kB 60.9 MB/s 
     |████████████████████████████████| 3.3 MB 58.0 MB/s 
     |████████████████████████████████| 596 kB 68.4 MB/s 
     |████████████████████████████████| 271 kB 72.3 MB/s 
     |████████████████████████████████| 160 kB 71.2 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16181 sha256=da2f8cbf07475da96dc08226bdf23a696d00527835070d268e78182612aa6633
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval
  Attempting uninstall:

##Imports

In [46]:
import datetime, os
import gradio as gr

from datasets import load_dataset

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import tensorflow as tf

import transformers
from transformers import AutoTokenizer, TFAutoModelForCausalLM, AdamWeightDecay, pipeline
print(transformers.__version__)

4.12.2


##Setup

In [24]:
datasets = load_dataset("wikitext", "wikitext-2-raw-v1")

Reusing dataset wikitext (/root/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/aa5e094000ec7afeb74c3be92c88313cd6f132d564c7effd961c10fd47c76f20)


  0%|          | 0/3 [00:00<?, ?it/s]

In [25]:
model_checkpoint = "distilgpt2"
block_size = 128
learning_rate = 2e-5
weight_decay = 0.01

##Preprocessing

In [26]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [27]:
def tokenize_function(examples):
    return tokenizer(examples["text"])

In [28]:
tokenized_datasets = datasets.map(
    tokenize_function, batched=True, num_proc=4, remove_columns=["text"]
    )

In [29]:
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported
    # it instead of this drop
    total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [30]:
lm_datasets = tokenized_datasets.map(
    group_texts,
    batched=True,
    batch_size=1000,
    num_proc=4,
)

##Model Definition

In [31]:
model = TFAutoModelForCausalLM.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/313M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at distilgpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [32]:
train_set = lm_datasets["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels"],
    shuffle=True,
    batch_size=16,
)
validation_set = lm_datasets["validation"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "labels"],
    shuffle=False,
    batch_size=16,
)

In [35]:
optimizer = AdamWeightDecay(lr=learning_rate, weight_decay_rate = weight_decay)

model.compile(optimizer=optimizer)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")
No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! Please ensure your labels are passed as the 'labels' key of the input dict so that they are accessible to the model during the forward pass. To disable this behaviour, please pass a loss argument, or explicitly pass loss=None if you do not want your model to compute a loss.


##Create pipeline for inference

In [42]:
language_model_pipeline = pipeline("text-generation", model = model, tokenizer = tokenizer)

In [43]:
def language_model_func(input):
  return language_model_pipeline(input)[0]["generated_text"]

In [47]:
gr.Interface(fn = language_model_func, inputs = "text", outputs = "text").launch()

Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
This share link will expire in 72 hours. If you need a permanent link, visit: https://gradio.app/introducing-hosted
Running on External URL: https://29790.gradio.app


(<Flask 'gradio.networking'>,
 'http://127.0.0.1:7860/',
 'https://29790.gradio.app')